---
title: "Publications"
echo: false
jupyter: python3
section-divs: false
format: 
  html: 
    theme: callout_highlights.scss
---

In [ ]:
# Courtesy of https://ddimmery.com/posts/quarto-website/
# Drew Dimmery. 2022. “Quarto for an Academic Website.” May 11, 2022. https://ddimmery.com/posts/quarto-website.
import yaml
from IPython.display import display, Markdown, HTML
from datetime import date
 
def readable_list(_s):
  if len(_s) < 3:
    return ' and '.join(map(str, _s))
  *a, b = _s
  return f"{', '.join(map(str, a))}, & {b}"
def button(url, str, icon):
    icon_base = icon[:2]
    return f"""<a class="btn btn-outline-dark btn-sm", href="{url}" target="_blank" rel="noopener noreferrer">
        <i class="{icon_base} {icon}" role='img' aria-label='{str}'></i>
        {str}
    </a>"""
def callout_highlight(highlight, id):
    return [
        f'''
        <!-- Create a button that toggles the visibility of content -->
        <button class="btn btn-primary btn-sm shadow-sm" type="button" data-bs-toggle="collapse" data-bs-target="#collapse-{id}" aria-expanded="false" aria-controls="collapse-{id}" aria-expanded="false" aria-label="Toggle callout">
            <i class="bi bi-pen-fill" role="img"></i>
            Research Highlights
        </button>
        ''',
        f'''
        <!-- Add the content that will be hidden/shown -->
        <div class="collapse" id="collapse-{id}">
                <div class="card card-body bg-light small">
                    {highlight}
            </div>
        </div>
        ''']
yaml_data = yaml.safe_load(open("publications.yml"))
pub_strs = {"pubs": {}, "wps": {}, "letters": {}, "thesis": {}}
for _, data in yaml_data.items():
    title_str = data["title"]
    authors = data.get("authors", ["me"])
    authors = [aut if aut != "me" else "<strong>Andreotta, M.</strong>" for aut in authors]
    author_str = readable_list(authors)
    year_str = data["year"]
    if year_str is None:
        year_str = date.today().year
    buttons = []
    preprint = data.get("preprint")
    if preprint is not None:
        buttons.append(button(preprint, "Preprint", "bi-file-earmark-pdf"))
    github = data.get("github")
    if github is not None:
        buttons.append(button(github, "Github", "bi-github"))
    osf = data.get("osf")
    if osf is not None:
        buttons.append(button(osf, "OSF", "bi-box"))
    pub_url = data.get("published_url")
    outlet = data.get("outlet")
    volume = data.get("volume")
    issue = data.get("issue")
    pages = data.get("pages")
    output = data.get("output")
    highlight = data.get("highlight")

    in_press = False
    if(isinstance(year_str, str)):
        in_press = "in-press" in year_str
    working_paper = ((pub_url is None) and not in_press) or output == "working"
    thesis = pub_url == "https://research-repository.uwa.edu.au/en/publications/mental-models-under-ideological-constraints-the-psychological-sig"
    pub_str = f'{author_str} ({year_str}). {title_str}.'
    if outlet is not None:
        pub_str += f" <em>{outlet}</em>"
        if volume is not None:
            pub_str += f", <em>{volume}</em>"
            if issue is not None:
                pub_str += f"({issue})"
        if pages is not None:
            pub_str += f", {pages}"
        pub_str += "."
            
    if working_paper:
        if year_str not in pub_strs["wps"]:
            pub_strs["wps"][year_str] = []
        pub_strs["wps"][year_str].append(
            "<li class='list-group-item'>" + pub_str + "<br>" + " ".join(buttons) + "</li>"
        )
    elif output == "letter":
        if in_press:
            year_str = date.today().year
        if year_str not in pub_strs["letters"]:
            pub_strs["letters"][year_str] = []
        if not in_press:
            buttons.append(button(pub_url, "Letter", "bi-file-text"))
        pub_strs["letters"][year_str].append(
            "<li class='list-group-item'>" + pub_str + "<br>" + " ".join(buttons) + "</li>"
        )        
    elif not thesis:
        if in_press:
            year_str = date.today().year
        if year_str not in pub_strs["pubs"]:
            pub_strs["pubs"][year_str] = []
        if not in_press:
            buttons.append(button(pub_url, "Publication", "bi-file-text"))
        if highlight is not None:
            highlight_str = [callout_highlight(highlight, _)[0], "<br> " + callout_highlight(highlight, _)[1]]
        else:
            highlight_str = ["", ""]
        pub_strs["pubs"][year_str].append(
            "<li class='list-group-item'>" + pub_str + "<br>" + highlight_str[0] + " ".join(buttons) + highlight_str[1] +"</li>"
        )
    elif thesis:
        if year_str not in pub_strs["thesis"]:
            pub_strs["thesis"][year_str] = []
        buttons.append(button(pub_url, "Online", "bi-file-text"))
        pub_strs["thesis"][year_str].append(
            "<li class='list-group-item'>" + pub_str + "<br>" + " ".join(buttons) + "</li>"
        )

## Peer-Reviewed Publications


In [ ]:
#| label: published-year
#| output: asis

this_year = date.today().year
all_years = [year for year in pub_strs["pubs"].keys() if isinstance(year, int)]

for year in sorted(pub_strs["pubs"].keys(), reverse=True):
    display(Markdown(f"### {year}" + "{#" + f"published-{year}" + "}"))
    display(HTML(
        "<ul class='list-group list-group-flush'>" + '\n'.join(pub_strs["pubs"][year]) + "</ul>"
    ))

## Working Papers


In [ ]:
#| label: not-published-year
#| output: asis
for year in sorted(pub_strs["wps"].keys(), reverse=True):
    display(Markdown(f"### {year}" + "{#" + f"not-published-{year}" + "}"))
    display(HTML(
        "<ul class='list-group list-group-flush'>" + '\n'.join(pub_strs["wps"][year]) + "</ul>"
    ))

## Letters to the Editor

In [ ]:
#| label: letters-year
#| output: asis
for year in sorted(pub_strs["letters"].keys(), reverse=True):
    display(HTML(
        "<ul class='list-group list-group-flush'>" + '\n'.join(pub_strs["letters"][year]) + "</ul>"
    ))

## PhD Thesis


In [ ]:
#| label: thesis-year
#| output: asis
for year in sorted(pub_strs["thesis"].keys(), reverse=True):
    display(HTML(
        "<ul class='list-group list-group-flush'>" + '\n'.join(pub_strs["thesis"][year]) + "</ul>"
    ))